In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
#changing the working directory
%cd /content/gdrive/MyDrive/Github/sigGCN-main/

/content/gdrive/MyDrive/Github/sigGCN-main


# BaronHuman

In [ ]:
!python siggcn.py --dirData="data/" --dataset="BaronHuman" --dirAdj="data/BaronHuman/" --dirLabel="data/BaronHuman/" --outputDir="data/output" --saveResults=0 --epochs=250 --batchsize=30

# Zhengsorted

In [ ]:
!python siggcn.py --dirData="data/" --dataset="Zhengsorted" --dirAdj="data/Zhengsorted/" --dirLabel="data/Zhengsorted/" --outputDir="data/output" --saveResults=0 --epochs=1


In [ ]:
%pycat train.py


In [ ]:
%%writefile train.py
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Mar 23 18:30:20 2021

@author: tianyu
"""
import time
import pandas as pd
import torch
#from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from sklearn import metrics, preprocessing
from sklearn.metrics import roc_curve, auc

import numpy as np
import sys
sys.path.insert(0, 'lib/')
import utilsdata

def calc_auc(test_labels, preds_probs, dataset):
  if (dataset == 'Zhengsorted'):
    classes = [0,1,2,3,4,5,6,7,8,9]
  elif (dataset == 'BaronHuman'):
    classes = [0,1,2,3,4,5,6,7,8,9,10,11,12,13]
  
  test_labels = preprocessing.label_binarize(test_labels.values, classes=classes)

  n_classes = test_labels.shape[1]
  print('n_classes: ', n_classes)

  fpr = dict()
  tpr = dict()
  roc_auc = dict()
  for i in range(n_classes):
      fpr[i], tpr[i], _ = roc_curve(test_labels[:, i], preds_probs[:, i])
      roc_auc[i] = auc(fpr[i], tpr[i])

  fpr["micro"], tpr["micro"], _ = roc_curve(test_labels.ravel(), preds_probs.ravel())
  roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

  return fpr, tpr, roc_auc


def calculation(pred_test, test_labels, preds_probs, dataset, method='GCN'):
    test_acc = metrics.accuracy_score(pred_test, test_labels)
    test_f1_macro = metrics.f1_score(pred_test, test_labels, average='macro')
    test_f1_micro = metrics.f1_score(pred_test, test_labels, average='micro')
    precision = metrics.precision_score(test_labels, pred_test, average='micro')
    recall = metrics.recall_score(test_labels, pred_test, average='micro')
    
    # print('test_labels', test_labels.shape)
    # print('preds_probs', preds_probs.shape)

    fpr, tpr, roc_auc = calc_auc(test_labels, preds_probs, dataset)

    # print('method','test_acc','f1_test_macro','f1_test_micro','Testprecision','Testrecall','Testauc')
    # print(method, test_acc, test_f1_macro, test_f1_micro, precision,recall, 'something')
    # print(metrics.confusion_matrix(test_labels, pred_test))
    # print(fpr)
    # print()
    # print(tpr)
    # print()
    # print(roc_auc)
    return fpr, tpr, roc_auc
        
def weight_init(m):
    if isinstance(m, torch.nn.Conv2d) or isinstance(m, torch.nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        if m.bias is not None: 
            m.bias.data.fill_(0.0)

def test_model(net, loader, L, args):
    t_start_test = time.time()
    
    net.eval()
    test_acc = 0
    count = 0
    confusionGCN = np.zeros([args.nclass, args.nclass])
    predictions = pd.DataFrame()
    y_true = []
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
    
#    for batch_x, batch_y in loader:
#        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
#
#        out_gae, out_hidden, pred, out_adj = net(batch_x, args.dropout, L)
#        
#        test_acc += utilsdata.accuracy(pred, batch_y).item()
#        count += 1
#        y_true.append(batch_y.item())
#        #y_pred.append(pred.max(1)[1].item())
#        confusionGCN[batch_y.item(), pred.max(1)[1].item()] += 1
#        px = pd.DataFrame(pred.detach().cpu().numpy())            
#        predictions = pd.concat((predictions, px),0)
#        
#    t_total_test = time.time() - t_start_test
#    preds_labels = np.argmax(np.asarray(predictions), 1)
#    test_acc = test_acc/count
#    predictions.insert(0, 'trueLabels', y_true)


    for batch_x, batch_y in loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)

        out_gae, out_hidden, pred, out_adj = net(batch_x, args.dropout, L)
        
        test_acc += utilsdata.accuracy(pred, batch_y).item() * len(batch_y)
        count += 1
        y_true = batch_y.detach().cpu().numpy()
        y_predProbs = pred.detach().cpu().numpy()
        
    predictions = pd.DataFrame(y_predProbs)            
    for i in range(len(y_true)):
        confusionGCN[y_true[i], np.argmax(y_predProbs[i,:])] += 1
    
    t_total_test = time.time() - t_start_test
    preds_labels = np.argmax(np.asarray(predictions), 1)
    preds_probs = np.asarray(predictions)
    test_acc = test_acc/len(loader.dataset)
    predictions.insert(0, 'trueLabels', y_true)
    n_classes = preds_probs.shape[1]
    
    return test_acc, confusionGCN, predictions, preds_labels, t_total_test, preds_probs, n_classes

        
def train_model(useModel, train_loader, val_loader, L, args):    

    # network parameters
    D_g = args.num_gene
    CL1_F = 5
    CL1_K = 5
    FC1_F = 32
    FC2_F = 0
    NN_FC1 = 256
    NN_FC2 = 32
    out_dim = args.nclass  
    net_parameters = [D_g, CL1_F, CL1_K, FC1_F,FC2_F,NN_FC1, NN_FC2, out_dim]

    # learning parameters
    dropout_value = 0.2
    l2_regularization = 5e-4
    batch_size = args.batchsize
    num_epochs = args.epochs
    

    nb_iter = int(num_epochs * args.train_size) // batch_size
    print('num_epochs=',num_epochs,', train_size=',args.train_size,', nb_iter=',nb_iter)
    
    # Optimizer
    global_lr = args.lr
    global_step = 0
    decay = 0.95
    decay_steps = args.train_size
        
        
   # instantiate the object net of the class
    net = useModel(net_parameters)
    net.apply(weight_init)
    
    if torch.cuda.is_available():
        net.cuda()
        
    print(net)
            
    #optimizer = optim.Adam(net.parameters(),lr= args.lr, weight_decay=5e-4)
    optimizer = optim.SGD(net.parameters(), momentum=0.9, lr= args.lr)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
    
    ## Train   
    net.train()
    losses_train = []
    acc_train = []
    
    t_total_train = time.time()

    def adjust_learning_rate(optimizer, epoch, lr):
        """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    #    lr = args.lr * (0.1 ** (epoch // 30))
        lr = lr * pow( decay , float(global_step// decay_steps) )
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
        return lr
    
    for epoch in range(num_epochs):  # loop over the dataset multiple times
    
        # update learning rate
        cur_lr = adjust_learning_rate(optimizer,epoch, args.lr)
        
        # reset time
        t_start = time.time()
    
        # extract batches
        epoch_loss = 0.0
        epoch_acc = 0.0
        count = 0
        for i, (batch_x, batch_y) in enumerate(train_loader):
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
                    
            optimizer.zero_grad()   
            out_gae, out_hidden, output, out_adj = net(batch_x, dropout_value, L)

            loss_batch = net.loss(out_gae, batch_x, output, batch_y, l2_regularization)
          
            acc_batch = utilsdata.accuracy(output, batch_y).item()
            
            loss_batch.backward()
            optimizer.step()
            
            count += 1
            epoch_loss += loss_batch.item()
            epoch_acc += acc_batch
            global_step += args.batchsize 
            
            # print
            if count % 1000 == 0: # print every x mini-batches
                print('epoch= %d, i= %4d, loss(batch)= %.4f, accuray(batch)= %.2f' % (epoch + 1, count, loss_batch.item(), acc_batch))
    
    
        epoch_loss /= count
        epoch_acc /= count
        losses_train.append(epoch_loss) # Calculating the loss
        acc_train.append(epoch_acc) # Calculating the acc
        # print
        t_stop = time.time() - t_start
        
        if epoch % 10 == 0 and epoch != 0:
            with torch.no_grad():
                val_acc = 0  
                count = 0
                for b_x, b_y in val_loader:
                    b_x, b_y = b_x.to(device), b_y.to(device)          
                    _, _, val_pred, _ = net(b_x, args.dropout, L)                    
                    val_acc += utilsdata.accuracy(val_pred, b_y).item() * len(b_y)
                    count += 1
                    
                val_acc = val_acc/len(val_loader.dataset)
                
            print('epoch= %d, loss(train)= %.3f, accuracy(train)= %.3f, time= %.3f, lr= %.5f' %
                  (epoch + 1, epoch_loss, epoch_acc, t_stop, cur_lr))
            print('----accuracy(val)= ', val_acc)
            print('training_time:',t_stop)
        else:
            print('epoch= %d, loss(train)= %.3f, accuracy(train)= %.3f, time= %.3f, lr= %.5f' %
                  (epoch + 1, epoch_loss, epoch_acc, t_stop, cur_lr))
            print('training_time:',t_stop)
        
    
    t_total_train = time.time() - t_total_train  
    
    return net, t_total_train


Overwriting train.py


In [ ]:
%%writefile siggcn.py
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Mar 22 14:00:37 2020

@author: tianyu
"""
   
import sys, os
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
import torch.utils.data as Data
import torch.optim as optim
import argparse
import time
import numpy as np

import scipy.sparse as sp
from scipy.sparse import csr_matrix


from sklearn.preprocessing import label_binarize

import pandas as pd
import sys
sys.path.insert(0, 'lib/')


if torch.cuda.is_available():
    print('cuda available')
    dtypeFloat = torch.cuda.FloatTensor
    dtypeLong = torch.cuda.LongTensor
    torch.cuda.manual_seed(1)
else:
    print('cuda not available')
    dtypeFloat = torch.FloatTensor
    dtypeLong = torch.LongTensor
    torch.manual_seed(1)

from coarsening import coarsen, laplacian
from coarsening import lmax_L
from coarsening import perm_data
from coarsening import rescale_L
from layermodel import *
import utilsdata
from utilsdata import *
from train import *
import warnings
warnings.filterwarnings("ignore")
#
#
# Directories.
parser = argparse.ArgumentParser()
parser.add_argument('--dirData', type=str, default='/Users/tianyu/Desktop/scRNAseq_Benchmark_datasets/Intra-dataset/', help="directory of cell x gene matrix")
parser.add_argument('--dataset', type=str, default='Zhengsorted', help="dataset")
parser.add_argument('--dirAdj', type = str, default = '/Users/tianyu/Desktop/scRNAseq_Benchmark_datasets/Intra-dataset/Zhengsorted/', help = 'directory of adj matrix')
parser.add_argument('--dirLabel', type = str, default = '/Users/tianyu/Desktop/scRNAseq_Benchmark_datasets/Intra-dataset/Zhengsorted/', help = 'directory of adj matrix')
parser.add_argument('--outputDir', type = str, default = 'data/output', help = 'directory to save results')
parser.add_argument('--saveResults', type=int, default = 0, help='whether or not save the results')

parser.add_argument('--normalized_laplacian', type=bool, default = True, help='Graph Laplacian: normalized.')
parser.add_argument('--lr', type=float, default = 0.01, help='learning rate.')
parser.add_argument('--num_gene', type=int, default = 1000, help='# of genes')
parser.add_argument('--epochs', type=int, default = 1, help='# of epoch')
parser.add_argument('--batchsize', type=int, default = 64, help='# of genes')
parser.add_argument('--dropout', type=float, default = 0.2, help='dropout value')
parser.add_argument('--id1', type=str, default = '', help='test in pancreas')
parser.add_argument('--id2', type=str, default = '', help='test in pancreas')

parser.add_argument('--net', type=str, default='String', help="netWork")
parser.add_argument('--dist', type=str, default='', help="dist type")
parser.add_argument('--sampling_rate', type=float, default = 1, help='# sampling rate of cells')

args = parser.parse_args()

t_start = time.process_time()


# Load data


print('load data...')    
adjall, alldata, labels, shuffle_index = utilsdata.load_largesc(path = args.dirData, dirAdj=args.dirAdj, dataset=args.dataset, net='String')

# generate a fixed shuffle index
if shuffle_index is not None:
    shuffle_index = shuffle_index.astype(np.int32)
else:
    shuffle_index = np.random.permutation(alldata.shape[0])
    np.savetxt(args.dirData +'/' + args.dataset +'/shuffle_index_'+args.dataset+'.txt')
    
train_all_data, adj = utilsdata.down_genes(alldata, adjall, args.num_gene)
L = [laplacian(adj, normalized=True)]


#####################################################

##Split the dataset into train, val, test dataset. Use a fixed shuffle index to fix the sample order for comparison.
train_data, val_data, test_data, train_labels, val_labels, test_labels = utilsdata.spilt_dataset(train_all_data, labels, shuffle_index)
args.nclass = len(np.unique(labels))
args.train_size = train_data.shape[0] 

# train_labels = label_binarize(train_labels, classes=[0,1,2,3,4,5,6,7,8,9])
# val_labels = label_binarize(val_labels, classes=[0,1,2,3,4,5,6,7,8,9])
# test_labels = label_binarize(test_labels, classes=[0,1,2,3,4,5,6,7,8,9])

## Use the train_data, val_data, test_data to generate the train, val, test loader
train_loader, val_loader, test_loader = utilsdata.generate_loader(train_data,val_data, test_data, 
                                                        train_labels, val_labels, test_labels, 
                                                        args.batchsize)




##Delete existing network if exists
try:
    del net
    print('Delete existing network\n')
except NameError:
    print('No existing network to delete\n')

# Train model
net, t_total_train = train_model(Graph_GCN, train_loader,val_loader, L, args)

## Val
val_acc,confusionGCN, predictions, preds_labels, t_total_test, preds_probs, n_classes = test_model(net, val_loader, L, args)
print('  accuracy(val) = %.3f %%, time= %.3f' % (val_acc, t_total_test))

# Test
test_acc,confusionGCN, predictions, preds_labels, t_total_test, preds_probs, n_classes = test_model(net, test_loader, L, args)
    
print('  accuracy(test) = %.3f %%, time= %.3f' % (test_acc, t_total_test))
fpr, tpr, roc_auc = calculation(preds_labels, predictions.iloc[:,0], preds_probs, args.dataset)
# calculation(preds_labels, predictions.iloc[:,0], preds_probs)

if args.saveResults:
    testPreds4save = pd.DataFrame(preds_labels,columns=['predLabels'])
    testPreds4save.insert(0, 'trueLabels', list(predictions.iloc[:,0]))
    confusionGCN = pd.DataFrame(confusionGCN)
    
    testPreds4save.to_csv(args.outputDir+'/gcn_test_preds_'+ args.dataset+ str(args.num_gene)+'.csv')
    predictions.to_csv(args.outputDir+'/gcn_testProbs_preds_'+ args.dataset+ str(args.num_gene) +'.csv')
    confusionGCN.to_csv(args.outputDir+'/gcn_confuMat_'+ args.dataset+ str(args.num_gene)+'.csv')    
    np.savetxt(args.outputDir+'/newgcn_train_time_'+args.dataset + str(args.num_gene) +'.txt', [t_total_train])   
    np.savetxt(args.outputDir+'/newgcn_test_time_'+args.dataset + str(args.num_gene) +'.txt', [t_total_test])

import matplotlib.pyplot as plt
import itertools
from matplotlib.font_manager import FontProperties

fontP = FontProperties()
fontP.set_size('x-small')

plt.figure()
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.3f})'
               ''.format(roc_auc["micro"]),
         color='r', linestyle=':', linewidth=2)

colors = itertools.cycle(['aqua', 'fuchsia', 'darkorange', 'indigo', 'yellow', 'darkgreen',
                          'teal', 'lime', 'brown', 'cadetblue', 'darkred', 'slategrey', 'purple', 'olive', 
                          'chocolate', 'crimson', 'lawngreen'])

if (args.dataset == 'Zhengsorted'):
  celltypes = ['CD14+ Monocyte', 'CD19+ B', 'CD34+ (Pos_label)', 'CD4+ T Helper2', 'CD4+/CD25 T Reg',
          'CD4+/CD45RA+/CD25- Naive T', 'CD4+/CD45RO+ Memory', 'CD56+ NK', 'CD8+ Cytotoxic T',
          'CD8+/CD45RA+ Naive Cytotoxic']
elif (args.dataset == 'BaronHuman'):
  celltypes = ['acinar', 'activated_stellate', 'alpha', 'beta', 'delta', 'ductal', 'endothelial', 'epsilon',
               'gamma', 'macrophage', 'mast', 'quiescent_stellate', 'schwann', 't_cell']

for i, color, cell in zip(range(n_classes), colors, celltypes):
    lw = 2
    plt.plot(fpr[i], tpr[i], color=color,
            lw=lw, label='{0} (area {1:0.3f})'
            .format(cell, roc_auc[i]))
# plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right", prop=fontP)
plt.savefig('roc_auc_{}.png'.format(args.dataset))


Overwriting siggcn.py


# New Section

In [ ]:
%%writefile lib/layermodel.py

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Jan 13 21:15:43 2020

@author: tianyu
"""
import torch
#from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
import sys
sys.path.insert(0, 'lib/')


if torch.cuda.is_available():
    print('cuda available')
    dtypeFloat = torch.cuda.FloatTensor
    dtypeLong = torch.cuda.LongTensor
    torch.cuda.manual_seed(1)
else:
    print('cuda not available')
    dtypeFloat = torch.FloatTensor
    dtypeLong = torch.LongTensor
    torch.manual_seed(1)

from coarsening import lmax_L
from coarsening import rescale_L
from utilsdata import sparse_mx_to_torch_sparse_tensor

class my_sparse_mm(torch.autograd.Function):
    """
    Implementation of a new autograd function for sparse variables,
    called "my_sparse_mm", by subclassing torch.autograd.Function
    and implementing the forward and backward passes.
    """

    @staticmethod
    def forward(ctx, W, x):  # W is SPARSE
        ctx.save_for_backward(W, x)
        y = torch.mm(W, x)
        return y

    @staticmethod
    def backward(ctx, grad_output):
        W, x = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input_dL_dW = torch.mm(grad_input, x.t())
        grad_input_dL_dx = torch.mm(W.t(), grad_input )
        return grad_input_dL_dW, grad_input_dL_dx




#########################################################################################################
class Graph_GCN(nn.Module):

    def __init__(self, net_parameters):

        print('Graph ConvNet: GCN')

        super(Graph_GCN, self).__init__()

        # parameters
        D_g, CL1_F, CL1_K,  FC1_F, FC2_F,NN_FC1, NN_FC2, out_dim = net_parameters
        CNN1_F, CNN1_K = 32, 5
        CL2_F, CL2_K = 10, 10
        D_nn = D_g
        self.in_dim = D_g
        self.out_dim = out_dim
        self.FC2_F = FC2_F
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.num_gene = D_nn
        self.initScale = initScale = 6
        self.poolsize = 8
        FC1Fin = CL1_F*(D_g//self.poolsize)
        self.FC1Fin = FC1Fin
        self.CL1_K = CL1_K; self.CL1_F = CL1_F; 
        
        # Feature_H, Feature_W = (Input_Height - filter_H + 2P)/S + 1, (Input_Width - filter_W + 2P)/S + 1
        height = int(np.ceil(np.sqrt(int(D_nn))))
        FC2Fin = int(CNN1_F * (height//2) ** 2)
        self.FC2Fin = FC2Fin;
        
        # graph CL1
        self.cl1 = nn.Linear(CL1_K, CL1_F)
#        # graph CL2
#        self.cl2 = nn.Linear(CL2_K*CL1_F, CL2_F)
#        #FC gcnpure
#        self.fc_gcnpure = nn.Linear(FC1Fin, self.out_dim)
        # FC 1
        self.fc1 = nn.Linear(FC1Fin, FC1_F)
        # FC 2
        if self.FC2_F == 0:
            FC2_F = self.num_gene
            print('---------',FC2_F)
        self.fc2 = nn.Linear(FC1_F, FC2_F)
        # FC 3
        self.fc3 = nn.Linear(FC2_F, D_g)
        # CNN_FC1
        self.cnn_fc1 = nn.Linear(FC2Fin, FC1_F)
        #FC_concat with CNN
        Fin = FC1Fin + FC2Fin; Fout = self.out_dim;
        self.FC_concat = nn.Linear(Fin, self.out_dim)             
        #FC_sum2 with NN
        Fin = FC1_F + NN_FC2; Fout = self.out_dim;
        self.FC_sum2 = nn.Linear(Fin, Fout)                  
        #FC_sum1 with CNN
        Fin = FC1_F + FC1_F; Fout = self.out_dim;
        self.FC_sum1 = nn.Linear(Fin, Fout)             
        # NN_FC1
        self.nn_fc1 = nn.Linear(self.in_dim, NN_FC1)
        # NN_FC2
        self.nn_fc2 = nn.Linear(NN_FC1, NN_FC2)
        # NN_FC3_decode
        self.nn_fc3 = nn.Linear(NN_FC2, NN_FC1)
        # NN_FC4_decode
        Fin = NN_FC2; Fout = self.in_dim;
        self.nn_fc4 = nn.Linear(Fin, Fout)        

        
        # nb of parameters
        nb_param = CL1_K* CL1_F + CL1_F          # CL1
#        nb_param += CL2_K* CL1_F* CL2_F + CL2_F  # CL2
        nb_param += FC1Fin* FC1_F + FC1_F        # FC1
#        nb_param += FC1_F* FC2_F + FC2_F         # FC2
        print('nb of parameters=',nb_param,'\n')


    def init_weights(self, W, Fin, Fout):

        scale = np.sqrt( self.initScale / (Fin+Fout) )
        W.uniform_(-scale, scale)
        
        return W


    def graph_conv_cheby(self, x, cl,L, Fout, K):

        # parameters
        # B = batch size
        # V = nb vertices
        # Fin = nb input features
        # Fout = nb output features
        # K = Chebyshev order & support size
        B, V, Fin = x.size(); B, V, Fin = int(B), int(V), int(Fin)

        # rescale Laplacian
        lmax = lmax_L(L)
        L = rescale_L(L, lmax)

        # convert scipy sparse matric L to pytorch
        L = sparse_mx_to_torch_sparse_tensor(L)
        if torch.cuda.is_available():
            L = L.cuda()

        # transform to Chebyshev basis
        x0 = x.permute(1,2,0).contiguous()  # V x Fin x B
        x0 = x0.view([V, Fin*B])            # V x Fin*B
        x = x0.unsqueeze(0)                 # 1 x V x Fin*B

        def concat(x, x_):
            x_ = x_.unsqueeze(0)            # 1 x V x Fin*B
            return torch.cat((x, x_), 0)    # K x V x Fin*B

        sparse_mm = my_sparse_mm.apply # added

        if K > 1:
            x1 = sparse_mm(L,x0)              # V x Fin*B
            x = torch.cat((x, x1.unsqueeze(0)),0)  # 2 x V x Fin*B
        for k in range(2, K):
            x2 = 2 * sparse_mm(L,x1) - x0
            x = torch.cat((x, x2.unsqueeze(0)),0)  # M x Fin*B --> K x V x Fin*B
            x0, x1 = x1, x2

        x = x.view([K, V, Fin, B])           # K x V x Fin x B
        x = x.permute(3,1,2,0).contiguous()  # B x V x Fin x K
        x = x.view([B*V, Fin*K])             # B*V x Fin*K

        # Compose linearly Fin features to get Fout features
        x = cl(x)                            # B*V x Fout
        x = x.view([B, V, Fout])             # B x V x Fout

        return x


    # Max pooling of size p. Must be a power of 2.
    def graph_max_pool(self, x, p):
        if p > 1:
            x = x.permute(0,2,1).contiguous()  # x = B x F x V
            x = nn.MaxPool1d(p)(x)             # B x F x V/p
            x = x.permute(0,2,1).contiguous()  # x = B x V/p x F
            return x
        else:
            return x


    def forward(self, x_in, d, L):
        
        x = x_in#[:,:self.num_gene]
        x_nn = x_in#[:,self.num_gene:]

        x = x.unsqueeze(2) # B x V x Fin=1
        x = self.graph_conv_cheby(x, self.cl1, L[0], self.CL1_F, self.CL1_K)

        x = F.relu(x)
        x = self.graph_max_pool(x, self.poolsize)

        # flatten()
        x = x.view(-1, self.FC1Fin)


        
        ##############################################
        ##                  GAE_re                  ##
        ##############################################
        x_reAdj = 0 #torch.stack([F.sigmoid(torch.mm(z_i, z_i.t())) for z_i in x_reAdj])
        
        ##############################################
        ##                  GAE                     ##
        ##############################################
        x = self.fc1(x)
        x = F.relu(x)
        x_hidden_gae = x

        x_decode_gae = self.fc2(x_hidden_gae)
#        x_decode_gae = F.relu(x_decode_gae)
        if self.FC2_F != 0:                
            x_decode_gae = F.relu(x_decode_gae)
            x_decode_gae  = nn.Dropout(d)(x_decode_gae)            
            x_decode_gae = self.fc3(x_decode_gae)            

        


        ##############################################
        ##                  GCN//NN                  ##
        ##############################################
        
      
        # NN
        x_nn = self.nn_fc1(x_nn) # B x V
        x_nn = F.relu(x_nn)
        x_nn = self.nn_fc2(x_nn)
        x_nn = F.relu(x_nn)

#        x_hidden_ae = x_nn
#        x_decode_ae = self.nn_fc3(x_hidden_ae)
#        x_decode_ae = F.relu(x_decode_ae)
#        x_decode_ae = self.nn_fc4(x_hidden_ae)
#        x_decode_ae = F.relu(x_decode_ae)       

        # concatenate layer  
        x = torch.cat((x_hidden_gae, x_nn),1)        
        x = self.FC_sum2(x)
        x = F.log_softmax(x)        

        return x_decode_gae, x_hidden_gae, x, x_reAdj


    def loss(self, y1, y_target1,y2, y_target2,l2_regularization):
     
        loss1 = nn.MSELoss()(y1, y_target1)
        loss2 = nn.NLLLoss()(y2,y_target2)            
        loss = 1 * loss1 + 1 * loss2 

        l2_loss = 0.0
        for param in self.parameters():
            data = param* param
            l2_loss += data.sum()


        loss += 0.2* l2_regularization* l2_loss

        return loss